<a href="https://colab.research.google.com/github/amhaiskar0921/AmazonProject/blob/main/FinalBERTModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Loading preprocessed CSV

In [2]:
import pandas as pd
import numpy as np

In [3]:
df_merged = pd.read_csv('/content/drive/MyDrive/Amazon (LA) - Multi-Class Product Classification (Team A)/Datasets/preprocessed_merged_data.csv')
df_merged.head(20)


,product_id,product_locale,product_title,product_description,product_brand,esci_label,query,bert_input
0,B00BO9BGYU,es,light irridiance oxigenada 20vol 1000ml l negr...,NaN,light irridiance,I,yunsey tinte amoniaco,[CLS] yunsey tinte amoniaco [SEP] light irridi...
1,3836582864,es,peter lindbergh fashion photography 40th anniv...,NaN,taschen,I,libro channel decoracion,[CLS] libro channel decoracion [SEP] peter lin...
2,B085TCJ87R,us,lenovo thinkcentre m75s small form factor sff ...,NaN,lenovo,I,ryzen 11,[CLS] ryzen 11 [SEP] lenovo thinkcentr m75 sma...
3,B002T3RV5S,us,delta faucet foundations single function tub s...,NaN,delta faucet,I,tub faucet without shower head,[CLS] tub faucet without shower head [SEP] del...
4,B082KFTF41,us,kokodi lcd writing tablet 10 inch colorful tod...,features love 1 10 inch lcd colorful screen of...,kokodi,E,educational toys 3 year old,[CLS] educ toy 3 year old [SEP] kokodi lcd wri...
5,B01N0DZF25,jp,小学館の図鑑neo 新版 動物,ご注意 お使いの端末によっては 一部読みづらい場合がございます お手持ちの端末で立ち読みファ...,NaN,I,学校に行きたくない君へ,[CLS] 学校 に 行き たく ない 君 へ [SEP] 小学 館 の 図鑑 neo 新版...
6,B01N0DZF25,jp,小学館の図鑑neo 新版 動物,ご注意 お使いの端末によっては 一部読みづらい場合がございます お手持ちの端末で立ち読みファ...,NaN,I,子なし のリアル,[CLS] 子 なし の リアル [SEP] 小学 館 の 図鑑 neo 新版 動物 [SE...
7,B084GXV474,us,tracfone carrier locked samsung galaxy j7 crow...,NaN,tracfone,E,android phone,[CLS] android phone [SEP] tracfon carrier lock...
8,B084GXV474,us,tracfone carrier locked samsung galaxy j7 crow...,NaN,tracfone,E,used phones,[CLS] use phone [SEP] tracfon carrier lock sam...
9,B08467FBSR,us,hind 4pk active workout athletic shorts girls ...,hind 4 pack athletic shorts girls perfect pack...,hind,S,soccer shorts girls,[CLS] soccer short girl [SEP] hind 4pk activ w...


##Changing e, s, c, i to 0, 1, 2, 3

In [4]:
label_map = {"E": 0, "S": 1, "C": 2, "I": 3}

# Assuming df_merged is your DataFrame containing the "esci_label" column
df_merged["esci_label"] = df_merged["esci_label"].map(label_map)

In [5]:
df_merged.head()

,product_id,product_locale,product_title,product_description,product_brand,esci_label,query,bert_input
0,B00BO9BGYU,es,light irridiance oxigenada 20vol 1000ml l negr...,NaN,light irridiance,3,yunsey tinte amoniaco,[CLS] yunsey tinte amoniaco [SEP] light irridi...
1,3836582864,es,peter lindbergh fashion photography 40th anniv...,NaN,taschen,3,libro channel decoracion,[CLS] libro channel decoracion [SEP] peter lin...
2,B085TCJ87R,us,lenovo thinkcentre m75s small form factor sff ...,NaN,lenovo,3,ryzen 11,[CLS] ryzen 11 [SEP] lenovo thinkcentr m75 sma...
3,B002T3RV5S,us,delta faucet foundations single function tub s...,NaN,delta faucet,3,tub faucet without shower head,[CLS] tub faucet without shower head [SEP] del...
4,B082KFTF41,us,kokodi lcd writing tablet 10 inch colorful tod...,features love 1 10 inch lcd colorful screen of...,kokodi,0,educational toys 3 year old,[CLS] educ toy 3 year old [SEP] kokodi lcd wri...


##train_test_split with 80% train, 20% validation

In [6]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00


In [7]:
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, DataCollatorWithPadding
from datasets import Dataset
import tensorflow as tf

In [8]:
# Split the DataFrame into training and validation sets
train_df, valid_df = train_test_split(df_merged, test_size=0.2, random_state=42)

##Tokenizing the train and validation Dataset objects

In [9]:
# Load the tokenizer and create a data collator
checkpoint = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [10]:
# Tokenize the datasets
tokenized_train_dataset = Dataset.from_pandas(train_df).map(lambda x: tokenizer(x['bert_input'], truncation=True), batched=True)
tokenized_valid_dataset = Dataset.from_pandas(valid_df).map(lambda x: tokenizer(x['bert_input'], truncation=True), batched=True)

Map:   0%|          | 0/4347 [00:00<?, ? examples/s]

Map:   0%|          | 0/1087 [00:00<?, ? examples/s]

##Creating TensorFlow dataset objects

In [11]:
# Convert Hugging Face datasets to TensorFlow datasets
tf_train_dataset = tokenized_train_dataset.to_tf_dataset(
    columns=["input_ids", "attention_mask", "token_type_ids"],
    label_cols=["esci_label"],
    batch_size=8,
    collate_fn=data_collator,
    shuffle=True
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:399: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [12]:
tf_validation_dataset = tokenized_valid_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["esci_label"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

##Loading the model

In [13]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=4)

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


##Baseline f1-score

In [14]:
preds = model.predict(tf_validation_dataset)["logits"]

136/136 [==============================] - 56s 193ms/step


In [15]:
class_preds = np.argmax(preds, axis=1)
print(preds.shape, class_preds.shape)

(1087, 4) (1087,)


In [16]:
from sklearn.metrics import f1_score

# Assuming true_labels is your ground truth labels for the validation set
true_labels = valid_df["esci_label"]

# Compute F1-score
f1 = f1_score(true_labels, class_preds, average='micro')

print("F1-score:", f1)

F1-score: 0.500459981600736


###Baseline f1-score: English

In [17]:
valid_df_english = valid_df[valid_df['product_locale'] == 'us']
tokenized_validation_dataset_en = Dataset.from_pandas(valid_df_english).map(lambda x: tokenizer(x['bert_input'], truncation=True), batched=True)

Map:   0%|          | 0/746 [00:00<?, ? examples/s]

In [18]:
tf_validation_dataset_en = tokenized_validation_dataset_en.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["esci_label"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:399: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [19]:
preds_en = model.predict(tf_validation_dataset_en)["logits"]

94/94 [==============================] - 17s 181ms/step


In [22]:
class_preds_en= np.argmax(preds_en, axis=1)
print(preds_en.shape, class_preds_en.shape)

(746, 4) (746,)


In [23]:
from sklearn.metrics import f1_score

# Assuming true_labels is your ground truth labels for the validation set
true_labels_en = valid_df_english["esci_label"]

# Compute F1-score
f1 = f1_score(true_labels_en, class_preds_en, average='micro')

print("F1-score:", f1)

F1-score: 0.5375335120643432


###Baseline f1-score: Spanish

In [24]:
valid_df_es = valid_df[valid_df['product_locale'] == 'es']
tokenized_validation_dataset_es = Dataset.from_pandas(valid_df_es).map(lambda x: tokenizer(x['bert_input'], truncation=True), batched=True)

Map:   0%|          | 0/142 [00:00<?, ? examples/s]

In [25]:
tf_validation_dataset_es = tokenized_validation_dataset_es.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["esci_label"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:399: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [26]:
preds_es = model.predict(tf_validation_dataset_es)["logits"]

18/18 [==============================] - 4s 209ms/step


In [27]:
class_preds_es= np.argmax(preds_es, axis=1)
print(preds_es.shape, class_preds_es.shape)

(142, 4) (142,)


In [28]:
from sklearn.metrics import f1_score

# Assuming true_labels is your ground truth labels for the validation set
true_labels_es = valid_df_es["esci_label"]

# Compute F1-score
f1 = f1_score(true_labels_es, class_preds_es, average='micro')

print("F1-score:", f1)

F1-score: 0.4014084507042253


###Baseline f1-score: Japanese

In [29]:
valid_df_jp = valid_df[valid_df['product_locale'] == 'jp']
tokenized_validation_dataset_jp = Dataset.from_pandas(valid_df_jp).map(lambda x: tokenizer(x['bert_input'], truncation=True), batched=True)

Map:   0%|          | 0/199 [00:00<?, ? examples/s]

In [30]:
tf_validation_dataset_jp = tokenized_validation_dataset_jp.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["esci_label"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:399: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [31]:
preds_jp = model.predict(tf_validation_dataset_jp)["logits"]

25/25 [==============================] - 6s 235ms/step


In [32]:
class_preds_jp= np.argmax(preds_jp, axis=1)
print(preds_jp.shape, class_preds_jp.shape)

(199, 4) (199,)


In [33]:
from sklearn.metrics import f1_score

# Assuming true_labels is your ground truth labels for the validation set
true_labels_jp = valid_df_jp["esci_label"]

# Compute F1-score
f1 = f1_score(true_labels_jp, class_preds_jp, average='micro')

print("F1-score:", f1)

F1-score: 0.4321608040201005


##Training the model

In [15]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

model.compile(
    optimizer="adam",
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [16]:
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
)

544/544 [==============================] - 412s 673ms/step - loss: 0.9754 - accuracy: 0.6501 - val_loss: 0.9655 - val_accuracy: 0.6394


##Saving the fine-tuned model to Drive

In [17]:
model.save('/content/drive/MyDrive/Amazon (LA) - Multi-Class Product Classification (Team A)/finetuned-model')

In [18]:
preds = model.predict(tf_validation_dataset)["logits"]

136/136 [==============================] - 30s 199ms/step


In [19]:
class_preds = np.argmax(preds, axis=1)
print(preds.shape, class_preds.shape)

(1087, 4) (1087,)


###Overall f1-score: finetuned

In [23]:
from sklearn.metrics import f1_score

# Assuming true_labels is your ground truth labels for the validation set
true_labels = valid_df["esci_label"]

# Compute F1-score
f1 = f1_score(true_labels, class_preds, average='micro')

print("F1-score:", f1)

F1-score: 0.6393744250229991


###English f1-score: finetuned

In [33]:
valid_df_english = valid_df[valid_df['product_locale'] == 'us']
tokenized_validation_dataset_en = Dataset.from_pandas(valid_df_english).map(lambda x: tokenizer(x['bert_input'], truncation=True), batched=True)

Map:   0%|          | 0/746 [00:00<?, ? examples/s]

In [34]:
tf_validation_dataset_en = tokenized_validation_dataset_en.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["esci_label"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:399: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [35]:
preds_en = model.predict(tf_validation_dataset_en)["logits"]

94/94 [==============================] - 17s 182ms/step


In [38]:
class_preds_en = np.argmax(preds_en, axis=1)
print(preds_en.shape, class_preds_en.shape)

(746, 4) (746,)


In [37]:
from sklearn.metrics import f1_score

# Assuming true_labels is your ground truth labels for the validation set
true_labels_en = valid_df_english["esci_label"]

# Compute F1-score
f1 = f1_score(true_labels_en, class_preds_en, average='micro')

print("F1-score:", f1)

F1-score: 0.6635388739946381


###Spanish f1-score: finetuned

In [39]:
valid_df_es = valid_df[valid_df['product_locale'] == 'es']
tokenized_validation_dataset_es = Dataset.from_pandas(valid_df_es).map(lambda x: tokenizer(x['bert_input'], truncation=True), batched=True)

Map:   0%|          | 0/142 [00:00<?, ? examples/s]

In [40]:
tf_validation_dataset_es = tokenized_validation_dataset_es.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["esci_label"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:399: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [44]:
preds_es = model.predict(tf_validation_dataset_es)["logits"]

18/18 [==============================] - 4s 210ms/step


In [45]:
class_preds_es= np.argmax(preds_es, axis=1)
print(preds_es.shape, class_preds_es.shape)

(142, 4) (142,)


In [46]:
from sklearn.metrics import f1_score

# Assuming true_labels is your ground truth labels for the validation set
true_labels_es = valid_df_es["esci_label"]

# Compute F1-score
f1 = f1_score(true_labels_es, class_preds_es, average='micro')

print("F1-score:", f1)

F1-score: 0.528169014084507


###Japanese f1-score: Finetuned

In [47]:
valid_df_jp = valid_df[valid_df['product_locale'] == 'jp']
tokenized_validation_dataset_jp = Dataset.from_pandas(valid_df_jp).map(lambda x: tokenizer(x['bert_input'], truncation=True), batched=True)

Map:   0%|          | 0/199 [00:00<?, ? examples/s]

In [48]:
tf_validation_dataset_jp = tokenized_validation_dataset_jp.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["esci_label"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:399: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [49]:
preds_jp = model.predict(tf_validation_dataset_jp)["logits"]

25/25 [==============================] - 6s 244ms/step


In [50]:
class_preds_jp= np.argmax(preds_jp, axis=1)
print(preds_jp.shape, class_preds_jp.shape)

(199, 4) (199,)


In [52]:
from sklearn.metrics import f1_score

# Assuming true_labels is your ground truth labels for the validation set
true_labels_jp = valid_df_jp["esci_label"]

# Compute F1-score
f1 = f1_score(true_labels_jp, class_preds_jp, average='micro')

print("F1-score:", f1)

F1-score: 0.628140703517588
